In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os

import time
import csv
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import plotly.graph_objects as go

In [2]:
training_dir = "D:/Datasets/Fish/processed/train/New folder/"
validation_dir = "D:/Datasets/Fish/processed/validation/New folder/"
test_dir = "D:/Datasets/Fish/processed/test/New folder/"

def count_files(path):
    Number_Of_Files=0
    for file in os.walk(path):
    #     print(file)
        for files in file[2]:
            Number_Of_Files=Number_Of_Files+1
    return Number_Of_Files

count_files(training_dir)

44956

In [3]:
# Split computation of targets into two loops. The first loop computes y_tilde (i.e. inter-frame differences). 2nd loop produces ground truth labels
# This function assumes the moving objects in the frame have a pixel intensity of 1 and the background is 0
n = 30  # the length of the time axis of the model's input tensor. 4n video frames are divided up to construct n tensor slices
h = 45  # 1.5s # of look-ahead frames. we average the consecutive frame difference over the h+1 next frames, including the current

def compute_targets(image_folder):
    T = count_files(image_folder)
    tc_start_frame = 4 * n - 1  # first frame inded in the valid range
    tc_end_frame = T - 2 - h  # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
    y_tilde = np.zeros(T)
    for t in range(T-1): #T-1
        pil_img_t0 = load_img(image_folder+f'{t:05}' + '.png', color_mode='grayscale', target_size=None)
        pil_img_t1 = load_img(image_folder+f'{t+1:05}' + '.png', color_mode='grayscale', target_size=None)
        
        img_array_0 = img_to_array(pil_img_t0, data_format='channels_last', dtype='uint8')
        img_array_1 = img_to_array(pil_img_t1, data_format='channels_last', dtype='uint8')
        #we must convert to float32 before performing subtraction, otherwise all negative values will be set as 255
        img_array_0 = img_array_0.astype(np.float32)/255
        img_array_1 = img_array_1.astype(np.float32)/255
        
        y_tilde[t] = np.sum(np.abs(img_array_1 - img_array_0))/np.prod(img_array_0.shape)#Also must normalize by numbr of pixels (later) using the target size in load_img
        
    # label each frame algorithmically
    b=0.1
    c=0.001
    y_label = np.zeros(T)
    for t in range(tc_start_frame, tc_end_frame+1):
        if np.ptp(y_tilde[t:t+h+1]) >= c and y_tilde[t]<=(np.min(y_tilde[t:t+h+1])+b*np.ptp(y_tilde[t:t+h+1])):
            y_label[t] = 1
        
    return y_tilde, y_label
        

In [4]:
# baseline predictor outputting frame-by-frame probabilities of positive
Y_label = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_label.npy')
Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
assert Y_label.shape == Y_tilde.shape

T = len(Y_label)  # number of images in the dataset
n = 30  # the length of the time axis of the model's input tensor. 4n video frames are divided up to construct n tensor slices
h = 10  # look-ahead. we average the consecutive frame difference over the h+1 next frames, including the current
d = 89 # baseline alg will look-back this many frames to make the prediction
tc_start_frame = 4 * n - 1  # first frame inded in the valid range
tc_end_frame = T - 2 - h  # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
assert tc_start_frame > d

def baseline_prediction_probabilistic(y_tilde):
    T = len(y_tilde)
    start_frame = tc_start_frame # first frame inded in the valid range
    end_frame = tc_end_frame # last frame index in the valid range, assuming images start at t=0 and go to t=T-1
    y_pred_baseline = np.zeros(T)
    b=0.2
    c=0.0006 # approx mean range of y_tilde across d time-steps
    gamma1 = 5300
    gamma2 = 11000
    center1 = c
    
    P1 = lambda x, c, gamma1: 1/(1 + np.exp(-gamma1*(x-c))) # input x is the range np.ptp(y_tilde[t-d:t]). higher range--> higher probability
    P2 = lambda x, low_point, gamma2: 1/(1 + np.exp(gamma2*(x-low_point))) # input x is y tilde at t-1. lower x-->higher probability
    
    for t in range(start_frame, end_frame+1):
        center2 = np.min(y_tilde[t-d:t])+b*np.ptp(y_tilde[t-d:t])
        y_pred_baseline[t] = (P1(np.ptp(y_tilde[t-d:t]), center1, gamma1) + P2(y_tilde[t-1], center2, gamma2))/2
    return y_pred_baseline

In [8]:
# Y_tilde, Y_label = compute_targets(training_dir)
# Y_tilde, Y_label = compute_targets(validation_dir)
# Y_tilde, Y_label = compute_targets(test_dir)

# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_label', Y_label)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_label', Y_label)
# np.save(r'D:\Datasets\Fish\processed\targets\test\Y_tilde', Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\test\Y_label', Y_label)

#make baseline predictions and save
# Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\validation\Y_tilde.npy')
# Y_label_baseline = baseline_prediction(Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\validation\Y_label_baseline', Y_label_baseline)

Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
Y_pred_baseline = baseline_prediction_probabilistic(Y_tilde)
Y_pred_baseline
np.save(r'D:\Datasets\Fish\processed\targets\test\Y_pred_baseline', Y_pred_baseline)

# Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\train\Y_tilde.npy')
# Y_label_baseline = baseline_prediction(Y_tilde)
# np.save(r'D:\Datasets\Fish\processed\targets\train\Y_label_baseline', Y_label_baseline)

In [14]:
# check variable shapes
Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
Y_label = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_label.npy')
Y_pred_baseline = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_pred_baseline.npy')
print("Y_tilde: {}, Y_label: {}, y_pred_baseline: {}".format(Y_tilde.shape[0], Y_label.shape[0], Y_pred_baseline.shape[0]))

Y_tilde: 5367, Y_label: 5367, y_pred_baseline: 5367


In [11]:
def pad_y_pred(Y, T, start, end):
    out = np.zeros(T)
    out[start:end] = Y
    return out

# crop data to valid range
def crop_to_valid_range(Y, n, h):
    T = len(Y)
    start_frame = 4 * n - 1  # first frame inded in the valid range
    end_frame = T - 2 - h
    return Y[start_frame:end_frame + 1]

In [21]:
# plot ytilde timeseries with Y_pred_baseline visualized

Y_tilde = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_tilde.npy')
Y_label = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_label.npy')
Y_pred_baseline = np.load(r'D:\Datasets\Fish\processed\targets\test\Y_pred_baseline.npy')
Y_pred_baseline = np.squeeze(Y_pred_baseline)

Y_tilde = crop_to_valid_range(Y_tilde, n, h)
Y_label = crop_to_valid_range(Y_label, n, h)
Y_pred_baseline = crop_to_valid_range(Y_pred_baseline, n, h)

y_baseline = Y_pred_baseline > 0.60378 # threshold maximizing youdens index
x = np.arange(0, len(Y_label), 1)


fig = go.Figure()

fig.add_trace(go.Scatter(name='ytilde',
                         x=x, y=Y_tilde,
                         mode='lines',

                         line=dict(
                             width=1,
                             color='gray'
                         ),
                         showlegend=False

                         ))


fig.add_trace(go.Scatter(name='baseline',
                         x=x, y=Y_tilde,
                         mode='markers',

                         marker=dict(
                             color=Y_pred_baseline,
                             size=10,
                             opacity=0.7,
                             colorscale="Jet",
                             colorbar=dict(thickness=20),
                             line=dict(
                                 color='black',
                                 width=1
                             )
                         ),

                         ))


fig.add_trace(go.Scatter(name='positive (baseline prediction)',
                         x=x[y_baseline == 1], y=Y_tilde[y_baseline == 1],
                         mode='markers',

                         marker=dict(
                             color='rgba(0, 206, 0, 0.0)',
                             size=13,
                             opacity=0.99,
                             symbol = 'square',
                             colorscale="Jet",
                             line=dict(
                                 color='black',
                                 width=0.6
                             )
                         ),

                         ))

fig.add_trace(go.Scatter(name='positive (ground truth)',
                         x=x[Y_label == 1], y=Y_tilde[Y_label == 1],
                         mode='markers',

                         marker=dict(
                             size=4,
                             color='black'
                         ),

                         ))

fig.update_layout(
    width=900,
    height=700,

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.67,
        font_size=14
    ),
    xaxis=dict(title_text='frames', title_font_size=18),
    yaxis=dict(title_text='\u1EF9', title_font_size=18)

)

fig.show()

In [9]:
# plot derivative of y tilde againts y tilde. 
Y_acc = np.diff(Y_tilde, n=1, axis=-1)
colsp = np.linspace(0,100,100)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(name='Y_acc',
                    x=Y_tilde[1:800], y=Y_acc[1:800],
                    mode='lines+markers',
                    
                    marker=dict(
                        size=6,
                        color=colsp
                    )    
))

fig2.show()

In [16]:
#Histogram
fig = go.Figure(data=[go.Histogram(x=Y_label, cumulative_enabled=False, histnorm='probability')])

fig.show()